# Sessie 4 - QR-ontbinding, eigenwaarden en -vectoren

Voer de onderstaande code uit.

In [ ]:
%matplotlib inline

import imageio.v3 as imageio
import matplotlib.pyplot as plt
import numpy as np
import os

## Image Compression

Image compression is een methode om een figuur op een meer compacte manier voor te stellen om zo irrelevante informatie uit de figuur te verwijderen.

Aangezien de singuliere waarden ontbinding (SWO) van een arbitraire matrix ($A = U \Sigma V'$) ons orthogonale (dus niet-gecorreleerde) stukken informatie ($U$ en $V'$) met bijhorende impactfactoren ($\Sigma$) teruggeeft, kunnen we de belangrijkste delen informatie behouden en de rest verwijderen door de kleinste impactfactoren op $0$ te zetten.

Dit heet de lage-rang approximatie van een figuur.

## 1. QR-iteratie

Implementeer de QR-iteratie methode.

In [ ]:
def qr_decomposition(A):
    """
    Compute the QR-decompositie of the matrix A.
    Output: Q, R
    """
    return np.linalg.qr(A)
    
def qr_iteration(A, n):
    """
    Run QR-iteratie on the matrix A for n iterations.
    Output: Q, R
    """
    Q = np.identity(len(A))
    for _ in range(n):
        Q0, R = qr_decomposition(A)
        A = np.dot(R,Q0) # A is gradually replaced by the upper triangular, giving the eigenvalues on the diagonal
        Q = np.dot(Q,Q0) # will contain the eigenvectors
    return Q, A

In [ ]:
### TEST ###

# Compose symmetric matrix A'A
A = np.asmatrix([[1, 2, 3],[4, 5, 6],[7, 8, 7]])
A = np.dot(A.transpose(),A)

# Perform QR-iteration
Q, R = qr_iteration(A, 1000)
print("A'A = "); print(A)
print("QRQ' = "); print(np.dot(np.dot(Q, R), Q.transpose()))

# R moet een bovendriehoeksmatrix zijn en de diagonaal van R moet de eigenwaardes van de matrix A bevatten
print("R = "); print(R)
print("Eigenwaardes van R = "); print(np.linalg.eig(A)[0])

## 2. Singuliere waarden ontbinding

Met behulp van QR-iteratie kan men de eigenwaarden en -vectoren berekenen van een <b>vierkante symmetrische</b> matrix $M$.
$$M = QRQ^T \Leftrightarrow MQ = QR$$
met $Q$ een unitaire matrix ($Q^T = Q^{-1}$) en $R$ een diagonaalmatrix met positieve elementen.<br>
De kolommen van $Q$ zijn de eigenvectoren, terwijl de diagonaalelementen van $R$ de bijhorende eigenwaarden geven.

Je kan met deze methode ook een singuliere waarden ontbinding (SWO) berekenen.<br>
Namelijk, $A = U \Sigma V^T$ met $U, V$ unitaire matrices en $\Sigma$ een diagonaal matrix met positieve elementen.

Implementeer een functie die de SWO berekent voor een matrix $A$ .

(Hint: Gegeven de matrix $A = U\Sigma V^T$, bereken $A^TA$. Wat kom je uit?)

<b>ANSWER:</b>

$$\begin{split}
A = U\Sigma V^T \Leftrightarrow A^TA &= V \Sigma^T U^T U \Sigma V^T\\
                                   &= V \Sigma U^{-1} U \Sigma V^T \quad \text{[$U$ is unitair, $\Sigma$ is diagonaal]}\\
                                   &= V \Sigma^2 V^T
\end{split}$$
Dit is exact wat we bekomen als we QR-iteratie toepassen op $A^TA$.

Dus, als we $A^TA = QRQ^T$ berekenen, hebben we $A = U\Sigma V^T$, waarbij:
* $V = Q$
* $\Sigma = \sqrt{R}$
* $U = AV\Sigma^{-1}$ $\ \ $ [$V$ is unitair, $\Sigma$ is diagonal with positive elements]

In [ ]:
def qr_svd(A):
    """
    Compute the Singular Value Decomposition (A = USV') using QR-iteration.
    Output: U, S, V
    """
    A2 = np.dot(A.transpose(), A)
    Q, R = qr_iteration(A2, 1000)
    print(R)
    
    V = Q
    S = np.diag(np.sqrt(np.diag(abs(R))))
    U = np.dot(np.dot(A, V), np.linalg.inv(S))
    return (U, S, V)

In [ ]:
### TEST ###

# Compose arbitrary matrix A
A = 10*np.asmatrix([[1, 2, 3],[4, 5, 6],[7, 8, 7]])

# Perform SVD
U, S, V = qr_svd(A)
print("A = "); print(A)
print("USV' = ", ); print(np.dot(np.dot(U, S), V.transpose()))
print("-----")
print("U = ", ); print(U) # unitaire matrix
print("S = ", ); print(S) # diagonaal matrix
print("V = ", ); print(V) # unitaire matrix

## 3. Lage-Rang Approximatie

Lage-rang approximatie is een methode om een matrix te benaderen door de rang te verlagen.
Aangezien de rang van een matrix gelijk is aan het aantal singuliere waarden die niet nul zijn, kunnen we onze qr_svd methode gebruiken om de SWO van de matrix te berekenen en enkel $k$ singuliere waarden (en vectoren) over te houden. Dit heet een k-rang approximatie van een matrix.

We kunnen dit gebruiken om bijvoorbeeld een figuur te comprimeren. Gegeven dat de $k$ hoogste singuliere waarden (met hun eigenvectoren) grotendeels de figuur bepalen, voer een k-rang approximatie uit op de figuur 'image.jpg' zodat zo weinig mogelijk informatie verloren gaat.

(Opmerking: Vergeet niet dat de eigen waarden in de matrix $R$ <b>niet</b> geordend zijn op grootte.)

In [ ]:
def load_image(input_file):
    """
    Load the image from the given input file as a grayscale matrix.
    """
    return np.asmatrix(imageio.imread(input_file, mode='F'))  # Use mode='F' for floating-point grayscale

def remove_smallest_values(M, k):
    """
    Set the smallest elements to zero on the diagonal of M.
    Only keep the k largest values.
    """
    M = np.diag(M)
    values = list(enumerate(M))
    values.sort(key=lambda x: x[1], reverse=True)
    values[k:] = list(map(lambda x: (x[0], 0), values[k:]))
    values.sort(key=lambda x: x[0])
    return np.diag(list(zip(*values))[1])

def compress_image(M, k):
    """
    Computes the k-rank approximation of matrix M.
    """
    U, S, V = qr_svd(M)
    S = remove_smallest_values(S, k)
    return np.dot(np.dot(U, S), V.transpose())

In [ ]:
### TEST ###

# Parameters
k = 5 #Choose a rank for your k-rank approximation
input_file = 'image.jpg'
output_file = 'compressed_image.jpg'

# Compress the image2
img = load_image(input_file)
compressed_img = compress_image(img, k)

# Plot the image
plt.figure(figsize=(20,10))
plt.subplot(121)
plt.imshow(img, cmap=plt.cm.gray)
plt.title('Original image')
plt.axis('off')
plt.subplot(122)
plt.imshow(compressed_img, cmap=plt.cm.gray)
plt.title(str(k) + '-Rank Approximation')
plt.axis('off')
plt.show()